Description/Info: 
- Input tm and pm info no manual feature selection
- Uses 23/24 & 22/23 data
- Uses L1,L2, Premiership


# 0. Imports

## 0.1 Modules & Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import shap

from xgboost import XGBRegressor

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor, AdaBoostRegressor 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import make_scorer, mean_squared_error, mean_squared_error, mean_absolute_error, r2_score, pairwise_distances
from sklearn.feature_selection import SelectKBest, f_regression, RFECV, SequentialFeatureSelector
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.decomposition import NMF

In [ ]:
pd.set_option("display.max_rows", None)   
pd.set_option("display.max_columns", None)   
pd.set_option("display.width", 0)            
pd.set_option("display.max_colwidth", None)

## 0.2 Data

In [ ]:
df_pm = pd.read_excel("""data file""")

In [ ]:
df_tm = pd.read_excel("""data file""")

In [ ]:
teams = set(df_pm["""player key"""].unique()) & set(df_tm["""team key"""].unique())

In [ ]:
df_pm = df_pm[df_pm["""player key"""].isin(teams)].copy()
df_tm = df_tm[df_tm["""team key"""].isin(teams)].copy()

# 1. Feature Engineering
Building the Vectors, NMF components and Compatibility Component

## 1.1 Feature Selection

In [ ]:
player_ft = [col for col in df_pm.columns if col != """target name""" and 'id' not in col]

In [ ]:
team_ft = [col for col in df_tm.columns if 'id' not in col]

## 1.2 Vector Creation

In [ ]:
player_vectors = df_pm.groupby("""target name""")[player_ft].mean().fillna(0)

In [ ]:
team_vectors = df_tm.groupby("""team key""")[team_ft].mean().fillna(0)

## 1.3 Scale

In [ ]:
player_scaler = MinMaxScaler()
team_scaler = MinMaxScaler()

In [ ]:
player_vectors_scaled = player_scaler.fit_transform(player_vectors)

In [ ]:
team_vectors_scaled = team_scaler.fit_transform(team_vectors)

## 1.4 Dimensionality Reduction (NMF)

In [ ]:
player_nmf = NMF(n_components=7, init='nndsvd', max_iter=2000)

In [ ]:
team_nmf = NMF(n_components=7, init='nndsvd', max_iter=2000)

In [ ]:
player_nmf_model = player_nmf.fit_transform(player_vectors_scaled)

In [ ]:
player_nmf_df = pd.DataFrame(player_nmf_model, index=player_vectors.index)

In [ ]:
player_nmf_components= pd.DataFrame(player_nmf.components_, columns=player_vectors.columns)

In [ ]:
player_nmf_components

In [ ]:
team_nmf_model = team_nmf.fit_transform(team_vectors_scaled)

In [ ]:
team_nmf_df = pd.DataFrame(team_nmf_model, index=team_vectors.index)

In [ ]:
team_nmf_components= pd.DataFrame(team_nmf.components_, columns=team_vectors.columns)

In [ ]:
team_nmf_components

## 1.5 Compatibility Calculation

In [ ]:
df_ft_eng = df_pm[["""match key""", """team key""", """target name""", """target name"""]].copy()

In [ ]:
df_ft_eng['player_nmf_vector'] = df_ft_eng["""target name"""].map(player_nmf_df.to_dict(orient='index'))
df_ft_eng['team_nmf_vector'] = df_ft_eng["""team key"""].map(team_nmf_df.to_dict(orient='index'))

In [ ]:
df_ft_eng['player_nmf_np'] = df_ft_eng['player_nmf_vector'].apply(lambda x: list(x.values()))
df_ft_eng['team_nmf_np'] = df_ft_eng['team_nmf_vector'].apply(lambda x: list(x.values()))

In [ ]:
# df_ft_eng['player_nmf_np'] = df_ft_eng['player_nmf_vector'].apply(lambda x: list(x.values()))
# df_ft_eng['team_nmf_np'] = df_ft_eng['team_nmf_vector'].apply(
#     lambda x: list(x.values()) if isinstance(x, dict) else [np.nan]*team_nmf_df.shape[1]
# )

### 1.5.1 Cosine Similarity

In [ ]:
df_ft_eng['cosine_style_compatibility_nmf'] = df_ft_eng.apply(
    lambda row: cosine_similarity([row['player_nmf_np']], [row['team_nmf_np']])[0][0],
    axis=1
)

In [ ]:
for col in player_nmf_df.columns:
    df_ft_eng[col] = df_ft_eng["""target name"""].map(player_nmf_df[col])

### 1.5.2 Manhatten

In [ ]:
df_ft_eng['style_distance_nmf_manhatten'] = df_ft_eng.apply(
    lambda row: pairwise_distances([row['player_nmf_np']], [row['team_nmf_np']], metric='manhattan')[0][0],
    axis=1
)

In [ ]:
df_ft_eng['style_similarity_nmf_manhattan'] = 1 - MinMaxScaler().fit_transform(
    df_ft_eng[['style_distance_nmf_manhatten']]
)

### 1.5.3 Euclidean 

In [ ]:
df_ft_eng['style_distance_nmf_euclidean'] = df_ft_eng.apply(
    lambda row: pairwise_distances([row['player_nmf_np']], [row['team_nmf_np']], metric='euclidean')[0][0],
    axis=1
)

In [ ]:
df_ft_eng['style_similarity_nmf_euclidean'] = 1 - MinMaxScaler().fit_transform(
    df_ft_eng[['style_distance_nmf_euclidean']]
)

## 1.6 Analysis of Score Results

In [ ]:
df_ft_eng[
    ['cosine_style_compatibility_nmf', 'style_similarity_nmf_manhattan', 'style_similarity_nmf_euclidean']
].corr()

In [ ]:
df_ft_eng[
    ['cosine_style_compatibility_nmf', 'style_similarity_nmf_manhattan', 'style_similarity_nmf_euclidean', """target name"""]
].corr()["""target name"""].sort_values(ascending=False)

In [ ]:
# And in the model I can do things like feature importance etc

## 1.7 New Final DF
Only includes ids and the new engineered features and the target

In [ ]:
new_col = [
    """match key""",
    """team key""",
    """target name""",
    """target name""",
    'cosine_style_compatibility_nmf',
    'style_similarity_nmf_manhattan',
    'style_similarity_nmf_euclidean'
]

In [ ]:
nmf_col = []
for col in df_ft_eng.columns:
    if str(col).isdigit():
        nmf_col.append(col)

In [ ]:
final_col = new_col + nmf_col

In [ ]:
df_final = df_ft_eng[final_col].copy()

In [ ]:
nmf_rename = {}
for col in nmf_col:
    nmf_rename[int(col)] = f'player_nmf_dim_{col}'

df_final = df_final.rename(columns=nmf_rename)

In [ ]:
df_final.head()

In [ ]:
# df_final.to_csv("df_final_nmf_4.csv", index=False)

# 2. Model Prep

## 2.1 Basic EDA

In [ ]:
features = pd.DataFrame(columns=['Features','Number of unique values','Number of nulls'])

for i, feat in enumerate(df_final.columns):
    features.loc[i] = [feat, df_final[feat].nunique(), df_final[feat].isnull().sum()]

features

In [ ]:
num_ft = df_final.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_ft = df_final.select_dtypes(include=["object", "bool"]).columns.tolist()
print(f"\n Numerical ft: {num_ft} \n Total = {len(num_ft)}")
print(f"\n Categorical ft: {cat_ft} \n Total = {len(cat_ft)}")

In [ ]:
correlations = df_final[num_ft].corr()
print("Correlations with Target:\n")
print(correlations["""target name"""].sort_values(ascending=False))

## 2.2 Pre processing

In [ ]:
df_final = df_final.drop(columns = ["""match key""","""team key""","""target name"""])

In [ ]:
df_final.head()

In [ ]:
num_ft = df_final.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_ft = df_final.select_dtypes(include=["object", "bool"]).columns.tolist()

In [ ]:
correlations = df_final[num_ft].corr()
print("Correlations with Target:\n")
print(correlations["""target name"""].sort_values(ascending=False))

### 2.2.1 Null values

In [ ]:
# num_imputer = Pipeline([
#     ('imputer', SimpleImputer(strategy='median')),
# ])

# cat_imputer = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent'))
# ])

# imputer = ColumnTransformer([
#     ('num', num_imputer, num_ft),
#     ('cat', cat_imputer, cat_ft)
# ])

# df_transformed = imputer.fit_transform(df_final)
# df_imputed = pd.DataFrame(df_transformed, index = df_final.index, columns=num_ft + cat_ft)
# df_imputed[num_ft] = df_imputed[num_ft].apply(pd.to_numeric)

In [ ]:
#Maybe do a trimmer for outliers but idt i need one
# trimmer = OutlierTrimmer(capping_method='quantiles', tail='both', fold=0.05)

# df_num_trimmed = trimmer.fit_transform(df_imputed[num_ft])
# trimmed_index = df_num_trimmed.index

# df_trimmed = df_imputed.loc[trimmed_index]


In [ ]:
# features = pd.DataFrame(columns=['Features','Number of unique values','Number of nulls'])

# for i, feat in enumerate(df_imputed.columns):
#     features.loc[i] = [feat, df_imputed[feat].nunique(), df_imputed[feat].isnull().sum()]

# features

### 2.2.2 Split Data

In [ ]:
target = """target name"""
X, y = df_final.drop(columns=[target]), df_final[target]

In [ ]:
X_train, X_test, y_train, y_test = (
    train_test_split(X, y, test_size=0.2, random_state=0)
)

In [ ]:
# num_features = [col for col in num_ft if col != """target name"""]

# num_transform = Pipeline([
#     ('scaler', StandardScaler())
# ])

# cat_transform = Pipeline([
#     ('encoder', OneHotEncoder(handle_unknown='ignore',sparse_output=False, min_frequency=0.05))
# ])

# preprocessor = ColumnTransformer([
#     ('num', num_transform, num_features),
#     ('cat', cat_transform, cat_ft)
# ])

#Dont need to scale as all the data comes from scaled vectors

In [ ]:
# X_train_scaled = preprocessor.fit_transform(X_train)

In [ ]:
# X_test_scaled = preprocessor.transform(X_test)

# 3. Automated Feature Selection

## 3.1 SelectKBest

In [ ]:
k_val = range(1, 11, 1)
kbest_results = []

for k in k_val:
    selector = SelectKBest(score_func=f_regression, k=k)
    X_kbest = selector.fit_transform(X_train, y_train)

    model = RandomForestRegressor()
    scores = cross_val_score(model, X_kbest, y_train, cv=5, scoring='r2')

    kbest_results.append((k, scores.mean()))

    selected_ft = X_train.columns[selector.get_support()]
    print(f"\nSelectKBest k={k}")
    print("Selected features:", selected_ft.tolist())
    print("Score avg results:")
    print (scores.mean(), scores.std())

In [ ]:
df_kbest = pd.DataFrame(kbest_results, columns=["k", "r2"])
df_kbest.plot(x="k", y="r2", kind="line", marker="o", title="SelectKBest Performance")
plt.show()

## 3.2 RFECV

In [ ]:
rf = RandomForestRegressor()

rfecv = RFECV(
    estimator=rf,
    step=1,
    cv=5,
    scoring='r2'
)

rfecv.fit(X_train, y_train)

selected_features_rfecv = X_train.columns[rfecv.support_]
print("Selected features (RFECV with R²):", selected_features_rfecv.tolist())
print(f"Optimal number of features: {rfecv.n_features_}")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(rfecv.cv_results_['mean_test_score']) + 1), rfecv.cv_results_['mean_test_score'], marker='o')
plt.xlabel("Number of Features Selected")
plt.ylabel("Cross-Validated R² Score")
plt.title("RFECV (Random Forest, R² Scoring)")
plt.grid(True)
plt.tight_layout()
plt.show()

## 3.3 SFS

In [ ]:
sfs_results = []
directions = ['forward', 'backward']
n_feat = range(1, 10, 1)

for d in directions: 
    for n in n_feat:
        model = RandomForestRegressor(random_state=3)

        sfs = SequentialFeatureSelector(
            estimator=model,
            n_features_to_select=n,
            direction=d,
        )
        
        sfs.fit(X_train, y_train)
        selected_feats = X_train.columns[sfs.get_support()]
        
        X_selected = X_train[selected_feats]
        scores = cross_val_score(model, X_selected, y_train, cv=5, scoring='r2')
        mean_score = scores.mean()
        std_score = scores.std()
        
        sfs_results.append({
            'Direction': d,
            'Num Features': n,
            'Mean R2 Score': mean_score,
            'Std Dev': std_score,
            'Features': selected_feats.tolist()
        })
        
sfs_df = pd.DataFrame(sfs_results)
print(sfs_df.sort_values(by='Mean R2 Score', ascending=False))

## 3.4 Evaluate

In [ ]:
#Make a new model for each with best params

#SelectKBest
k_selector = SelectKBest(score_func=f_regression, k=9) 
X_train_kbest = k_selector.fit_transform(X_train, y_train)
X_test_kbest = k_selector.transform(X_test)
selected_kbest_features = X_train.columns[k_selector.get_support()]

model_kbest = RandomForestRegressor()
model_kbest.fit(X_train_kbest, y_train)

In [ ]:
#RFECV
rfe_selector = RFECV(estimator=RandomForestRegressor(), step=1, cv=5, scoring='r2')
rfe_selector.fit(X_train, y_train)
selected_rfecv_features = X_train.columns[rfe_selector.support_]

X_train_rfe = X_train[selected_rfecv_features]
X_test_rfe = X_test[selected_rfecv_features]

model_rfe = RandomForestRegressor(random_state=42)
model_rfe.fit(X_train_rfe, y_train)

In [ ]:
#SFS
sfs_selector = SequentialFeatureSelector(
    estimator=RandomForestRegressor(),
    n_features_to_select=8, #Change
    direction= 'backward', #Change
)
sfs_selector.fit(X_train, y_train)
selected_sfs_features = X_train.columns[sfs_selector.get_support()]

X_train_sfs = X_train[selected_sfs_features]
X_test_sfs = X_test[selected_sfs_features]

model_sfs = RandomForestRegressor()
model_sfs.fit(X_train_sfs, y_train)

In [ ]:
for name, model, X_test in [('SelectKBest', model_kbest, X_test_kbest), ('RFE', model_rfe, X_test_rfe), ('SFS', model_sfs, X_test_sfs)]:
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    print(f"{name} MAE: {mae:.2f}, R^2: {r2:.3f}")

## 3.5 Final Selector

In [ ]:
selector = RFECV(estimator=RandomForestRegressor(), step=1, cv=5, scoring='r2')

selector.fit(X_train, y_train)
selected_features = X_train.columns[selector.get_support()]

X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

selected_features

# 4. Tree Ensembles

## 4.1 Random Forest

#### Train & Tune

##### Random Search (Narrows down the area of the parameter to look at)

In [ ]:
rfr = RandomForestRegressor(random_state = 0)

In [ ]:
rfr_grid = {
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],       
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None], 
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt','log2'],
    'bootstrap': [True, False]
}

In [ ]:
rfr_random_search = RandomizedSearchCV(
    estimator=rfr,
    param_distributions=rfr_grid,
    n_iter=50,
    cv=5,
    scoring='r2',
    verbose=1,
    n_jobs=-1,
    random_state=0
)

In [ ]:
rfr_random_search.fit(X_train_selected, y_train)
best_rfr = rfr_random_search.best_estimator_
print("Best parameters:", rfr_random_search.best_params_)
print("Best CV R²:", rfr_random_search.best_score_)

In [ ]:
rf_results_df = pd.DataFrame(rfr_random_search.cv_results_)
rf_results_df = rf_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
rf_results_df = rf_results_df.sort_values(by='mean_test_score', ascending=False)
rf_results_df.head(10)

##### Grid Search with Cross Validation (Narrows down after Random Search)

In [ ]:
rfr_grid_2 = {
    'n_estimators': [1500, 1600, 1700],       
    'max_depth': [15, 20, 25],
    'min_samples_split': [9, 10, 12, 15],
    'min_samples_leaf': [4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True]
}

In [ ]:
rfr_grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=1),
    param_grid=rfr_grid_2,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)
rfr_grid_search.fit(X_train_selected, y_train)
print("Best Parameters:", rfr_grid_search.best_params_)
print("Best R2 (CV):", rfr_grid_search.best_score_)

In [ ]:
best_rfr_2 = rfr_grid_search.best_estimator_

#### Test & Evaluate

In [ ]:
rfr_y_pred = best_rfr_2.predict(X_test_selected)

In [ ]:
score = best_rfr_2.score(X_test_selected, y_test)
mse = mean_squared_error(y_test, rfr_y_pred)
mae = mean_absolute_error(y_test, rfr_y_pred)
r2 = r2_score(y_test, rfr_y_pred)
rmse = np.sqrt(mse)

print("Model Evaluation Metrics:")
print(f"Score: {score:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

In [ ]:
#Actual vs predicted
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=rfr_y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="Perfect Fit")
plt.title("Actual vs Predicted Values")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.legend()
plt.show()

In [ ]:
#Residual histogram
residuals = y_test - rfr_y_pred

plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, bins=30, color="blue")
plt.title("Residual Distribution")
plt.xlabel("Residuals (Actual - Predicted)")
plt.ylabel("Frequency")
plt.axvline(0, color="red", linestyle="--", label="No Error")
plt.legend()
plt.show()

In [ ]:
#Biggest error list
errors_df = pd.DataFrame({
    "Actual": y_test,
    "Predicted": rfr_y_pred,
    "Residual": residuals
})

largest_errors = errors_df.sort_values(by="Residual", key=abs, ascending=False).head(10)
print("Top 10 Instances with Largest Errors:")
print(largest_errors)

## 4.2 Boosted Trees

### 4.2.1 Gradient Boosting

#### Train & Tune

##### Random Search (Narrows down the area of the parameter to look at)

In [ ]:
gbr = GradientBoostingRegressor(random_state = 0)

In [ ]:
gbr_grid = {
    'n_estimators': [100, 300, 500, 700, 1000],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'subsample': [0.5, 0.7, 1.0]
}

In [ ]:
gbr_random_search = RandomizedSearchCV(
    estimator=gbr,
    param_distributions=gbr_grid,
    n_iter=50,
    cv=5,
    scoring='r2',
    verbose=1,
    n_jobs=-1,
    random_state=0
)

In [ ]:
gbr_random_search.fit(X_train_selected, y_train)
best_gbr = gbr_random_search.best_estimator_
print("Best parameters:", gbr_random_search.best_params_)
print("Best CV R²:", gbr_random_search.best_score_)

In [ ]:
gbr_results_df = pd.DataFrame(gbr_random_search.cv_results_)
gbr_results_df = gbr_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
gbr_results_df = gbr_results_df.sort_values(by='mean_test_score', ascending=False)
gbr_results_df.head(10)


##### Grid Search with Cross Validation (Narrows down after Random Search)

In [ ]:
gbr_grid_2 = {
    'n_estimators': [50,100,250],
    'learning_rate': [0.01, 0.05],
    'max_depth': [10,12],
    'min_samples_split': [5,10],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2'],
}

In [ ]:
gbr_grid_search = GridSearchCV(
    estimator= GradientBoostingRegressor(random_state = 1),
    param_grid=gbr_grid_2,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)
gbr_grid_search.fit(X_train_selected, y_train)
print("Best Parameters:", gbr_grid_search.best_params_)
print("Best R2 (CV):", gbr_grid_search.best_score_)

In [ ]:

best_gbr_2 = gbr_grid_search.best_estimator_

#### Test & Evaluate

In [ ]:
gbr_y_pred = best_gbr.predict(X_test_selected)

In [ ]:
score = best_gbr.score(X_test_selected, y_test)
mse = mean_squared_error(y_test, gbr_y_pred)
mae = mean_absolute_error(y_test, gbr_y_pred)
r2 = r2_score(y_test, gbr_y_pred)
rmse = np.sqrt(mse)

print("Model Evaluation Metrics:")
print(f"Score: {score:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

In [ ]:
#Actual vs predicted
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=gbr_y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="Perfect Fit")
plt.title("Actual vs Predicted Values")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.legend()
plt.show()

In [ ]:
#Residual histogram
residuals = y_test - gbr_y_pred

plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, bins=30, color="blue")
plt.title("Residual Distribution")
plt.xlabel("Residuals (Actual - Predicted)")
plt.ylabel("Frequency")
plt.axvline(0, color="red", linestyle="--", label="No Error")
plt.legend()
plt.show()

In [ ]:
#Biggest error list
errors_df = pd.DataFrame({
    "Actual": y_test,
    "Predicted": gbr_y_pred,
    "Residual": residuals
})

largest_errors = errors_df.sort_values(by="Residual", key=abs, ascending=False).head(10)
print("Top 10 Instances with Largest Errors:")
print(largest_errors)

### 4.2.2 Histogram Gradient Boosting

#### Train & Tune

##### Random Search (Narrows down the area of the parameter to look at)

In [ ]:
hgb = HistGradientBoostingRegressor(random_state=0)

In [ ]:
hgb_grid = {
    'learning_rate': [0.01, 0.1, 0.2, 1],
    'max_iter': [100, 300, 500, 700, 1000],
    'max_depth': [None, 10, 20, 30],
    'max_leaf_nodes': [None, 10, 20, 30, 40],
    'min_samples_leaf': [10, 20, 30, 50]
}

In [ ]:
hgb_random_search = RandomizedSearchCV(
    estimator=hgb,
    param_distributions=hgb_grid,
    n_iter=50,
    cv=5,
    scoring='r2',
    verbose=1,
    n_jobs=-1,
    random_state=0
)

In [ ]:
hgb_random_search.fit(X_train_selected, y_train)
best_hgb = hgb_random_search.best_estimator_
print("Best parameters:", hgb_random_search.best_params_)
print("Best CV R²:", hgb_random_search.best_score_)

In [ ]:
hgb_results_df = pd.DataFrame(hgb_random_search.cv_results_)
hgb_results_df = hgb_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
hgb_results_df = hgb_results_df.sort_values(by='mean_test_score', ascending=False)
hgb_results_df.head(10)


##### Grid Search with Cross Validation (Narrows down after Random Search)

In [ ]:
hgb_grid_2 = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_iter': [100, 300, 500, 700, 1000],
    'max_depth': [None, 10, 20, 30],
    'min_samples_leaf': [10, 20, 30, 50],
    'l2_regularization': [0.0, 0.1, 0.5, 1.0],
    'max_bins': [255, 512, 1024]
}

In [ ]:
hgb_grid_search = GridSearchCV(
    estimator= HistGradientBoostingRegressor(random_state=1),
    param_grid=hgb_grid_2,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)
hgb_grid_search.fit(X_train_selected, y_train)
print("Best Parameters:", hgb_grid_search.best_params_)
print("Best R2 (CV):", hgb_grid_search.best_score_)

In [ ]:

best_hgb_2 = hgb_grid_search.best_estimator_

#### Test & Evaluate

In [ ]:
hgb_y_pred = best_hgb.predict(X_test_selected)

In [ ]:
score = best_hgb.score(X_test_selected, y_test)
mse = mean_squared_error(y_test, hgb_y_pred)
mae = mean_absolute_error(y_test, hgb_y_pred)
r2 = r2_score(y_test, hgb_y_pred)
rmse = np.sqrt(mse)

print("Model Evaluation Metrics:")
print(f"Score: {score:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

In [ ]:
#Actual vs predicted
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=hgb_y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="Perfect Fit")
plt.title("Actual vs Predicted Values")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.legend()
plt.show()

In [ ]:
#Residual histogram
residuals = y_test - hgb_y_pred

plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, bins=30, color="blue")
plt.title("Residual Distribution")
plt.xlabel("Residuals (Actual - Predicted)")
plt.ylabel("Frequency")
plt.axvline(0, color="red", linestyle="--", label="No Error")
plt.legend()
plt.show()

In [ ]:
#Biggest error list
errors_df = pd.DataFrame({
    "Actual": y_test,
    "Predicted": hgb_y_pred,
    "Residual": residuals
})

largest_errors = errors_df.sort_values(by="Residual", key=abs, ascending=False).head(10)
print("Top 10 Instances with Largest Errors:")
print(largest_errors)

### 4.2.3 XGBoost

#### Train & Tune

##### Random Search (Narrows down the area of the parameter to look at)

In [ ]:
xgb = XGBRegressor(random_state=0, verbosity=0)

In [ ]:
xgb_grid = {
    'n_estimators': [100, 300, 500, 700, 1000],
    'learning_rate': [0.2, 0.1, 0.05, 0.01],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.5, 0.7, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.7, 0.9, 1.0]
}

In [ ]:
xgb_random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=xgb_grid,
    n_iter=50,
    cv=5,
    scoring='r2',
    verbose=1,
    n_jobs=-1,
    random_state=0
)

In [ ]:
xgb_random_search.fit(X_train_selected, y_train)
best_xgb = xgb_random_search.best_estimator_
print("Best parameters:", xgb_random_search.best_params_)
print("Best CV R²:", xgb_random_search.best_score_)

In [ ]:
xgb_results_df = pd.DataFrame(xgb_random_search.cv_results_)
xgb_results_df = xgb_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
xgb_results_df = xgb_results_df.sort_values(by='mean_test_score', ascending=False)
xgb_results_df.head(10)


##### Grid Search with Cross Validation (Narrows down after Random Search)

In [ ]:
xgb_grid_2 = {
    'n_estimators': [300, 400, 500],
    'learning_rate': [0.05, 0.01],
    'max_depth': [3, 4, 5],
    'subsample': [0.9, 1.0],
    'colsample_bytree': [0.7]
}

In [ ]:
xgb_grid_search = GridSearchCV(
    estimator= XGBRegressor(random_state=1),
    param_grid=xgb_grid_2,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)
xgb_grid_search.fit(X_train_selected, y_train)
print("Best Parameters:", xgb_grid_search.best_params_)
print("Best R2 (CV):", xgb_grid_search.best_score_)

In [ ]:

best_xgb_2 = xgb_grid_search.best_estimator_

#### Test & Evaluate

In [ ]:
xgb_y_pred = best_xgb.predict(X_test_selected)

In [ ]:
score = best_xgb.score(X_test_selected, y_test)
mse = mean_squared_error(y_test, xgb_y_pred)
mae = mean_absolute_error(y_test, xgb_y_pred)
r2 = r2_score(y_test, xgb_y_pred)
rmse = np.sqrt(mse)

print("Model Evaluation Metrics:")
print(f"Score: {score:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

In [ ]:
#Actual vs predicted
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=xgb_y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="Perfect Fit")
plt.title("Actual vs Predicted Values")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.legend()
plt.show()

In [ ]:
#Residual histogram
residuals = y_test - xgb_y_pred

plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, bins=30, color="blue")
plt.title("Residual Distribution")
plt.xlabel("Residuals (Actual - Predicted)")
plt.ylabel("Frequency")
plt.axvline(0, color="red", linestyle="--", label="No Error")
plt.legend()
plt.show()

In [ ]:
#Biggest error list
errors_df = pd.DataFrame({
    "Actual": y_test,
    "Predicted": xgb_y_pred,
    "Residual": residuals
})

largest_errors = errors_df.sort_values(by="Residual", key=abs, ascending=False).head(10)
print("Top 10 Instances with Largest Errors:")
print(largest_errors)

### 4.2.4 AdaBoost

#### Train & Tune

##### Random Search (Narrows down the area of the parameter to look at)

In [ ]:
ada = AdaBoostRegressor(random_state=0)

In [ ]:
ada_grid = {
    'n_estimators': [50, 100, 200, 300, 400],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 1.0],
    'loss': ['linear', 'square', 'exponential']
}

In [ ]:
ada_random_search = RandomizedSearchCV(
    estimator=ada,
    param_distributions=ada_grid,
    n_iter=50,
    cv=5,
    scoring='r2',
    verbose=1,
    n_jobs=-1,
    random_state=0
)

In [ ]:
ada_random_search.fit(X_train_selected, y_train)
best_ada = ada_random_search.best_estimator_
print("Best parameters:", ada_random_search.best_params_)
print("Best CV R²:", ada_random_search.best_score_)

In [ ]:
ada_results_df = pd.DataFrame(ada_random_search.cv_results_)
ada_results_df = ada_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
ada_results_df = ada_results_df.sort_values(by='mean_test_score', ascending=False)
ada_results_df.head(10)


##### Grid Search with Cross Validation (Narrows down after Random Search)

In [ ]:
ada_grid_2 = { 
    'n_estimators': [50, 100, 200, 300, 400],
    'learning_rate': [0.005, 0.01, 0.05],
    'loss': ['linear']
}

In [ ]:
ada_grid_search = GridSearchCV(
    estimator= AdaBoostRegressor(random_state=1),
    param_grid=ada_grid_2,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)
ada_grid_search.fit(X_train_selected, y_train)
print("Best Parameters:", ada_grid_search.best_params_)
print("Best R2 (CV):", ada_grid_search.best_score_)

In [ ]:

best_ada_2 = ada_grid_search.best_estimator_

#### Test & Evaluate

In [ ]:
ada_y_pred = best_ada_2.predict(X_test_selected)

In [ ]:
score = best_ada_2.score(X_test_selected, y_test)
mse = mean_squared_error(y_test, ada_y_pred)
mae = mean_absolute_error(y_test, ada_y_pred)
r2 = r2_score(y_test, ada_y_pred)
rmse = np.sqrt(mse)

print("Model Evaluation Metrics:")
print(f"Score: {score:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

In [ ]:
#Actual vs predicted
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=ada_y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="Perfect Fit")
plt.title("Actual vs Predicted Values")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.legend()
plt.show()

In [ ]:
#Residual histogram
residuals = y_test - ada_y_pred

plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, bins=30, color="blue")
plt.title("Residual Distribution")
plt.xlabel("Residuals (Actual - Predicted)")
plt.ylabel("Frequency")
plt.axvline(0, color="red", linestyle="--", label="No Error")
plt.legend()
plt.show()

In [ ]:
#Biggest error list
errors_df = pd.DataFrame({
    "Actual": y_test,
    "Predicted": ada_y_pred,
    "Residual": residuals
})

largest_errors = errors_df.sort_values(by="Residual", key=abs, ascending=False).head(10)
print("Top 10 Instances with Largest Errors:")
print(largest_errors)

# 5. Ensemble of Ensembles

## 5.1  New Basic Models
Retraining basic models to avoid leaks

In [ ]:
rfr_model_ensembles = RandomForestRegressor(n_estimators= 2000, min_samples_split= 2, min_samples_leaf= 1, max_features= 'sqrt', max_depth= 10, bootstrap= True)
rfr_model_ensembles.fit(X_train_selected,y_train)

In [ ]:
gbr_model_ensembles = GradientBoostingRegressor(subsample= 1.0, n_estimators= 100, min_samples_split= 10, min_samples_leaf= 1, max_features= None, max_depth= 7, learning_rate= 0.05)
gbr_model_ensembles.fit(X_train_selected,y_train)

In [ ]:
hgb_model_ensembles = HistGradientBoostingRegressor(min_samples_leaf= 10, max_leaf_nodes= 30, max_iter= 300, max_depth= 20, learning_rate= 0.1)

hgb_model_ensembles.fit(X_train_selected,y_train)

In [ ]:
xgb_model_ensembles = XGBRegressor(colsample_bytree= 0.7, learning_rate= 0.05, max_depth= 5, n_estimators= 500, subsample= 1.0)
xgb_model_ensembles.fit(X_train_selected,y_train)

In [ ]:
ada_model_ensembles = AdaBoostRegressor(learning_rate= 0.005, loss= 'linear', n_estimators= 300)
ada_model_ensembles.fit(X_train_selected,y_train)

## 5.2 Voting 

### Training & Tuning

In [ ]:
voting_model = VotingRegressor(
    [
        ("gbr", gbr_model_ensembles),
        ("rfr", rfr_model_ensembles),
        ('hgb', hgb_model_ensembles), 
        ('xgb', xgb_model_ensembles),
        ('ada', ada_model_ensembles) 
        
    ]
)
voting_model.fit(X_train_selected, y_train)

In [ ]:
voting_score = cross_val_score(voting_model, X_train_selected, y_train, scoring=make_scorer(r2_score), cv=5)
print("Cross-validated R²:", voting_score.mean())

### Test & Evaluate

In [ ]:
voting_pred = voting_model.predict(X_test_selected)

In [ ]:
score = voting_model.score(X_test_selected, y_test)
mse = mean_squared_error(y_test, voting_pred)
mae = mean_absolute_error(y_test, voting_pred)
r2 = r2_score(y_test, voting_pred)
rmse = np.sqrt(mse)

print("Model Evaluation Metrics:")
print(f"Score: {score:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

In [ ]:
#Actual vs predicted
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=voting_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="Perfect Fit")
plt.title("Actual vs Predicted Values")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.legend()
plt.show()

In [ ]:
#Residual histogram
residuals = y_test - voting_pred

plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, bins=30, color="blue")
plt.title("Residual Distribution")
plt.xlabel("Residuals (Actual - Predicted)")
plt.ylabel("Frequency")
plt.axvline(0, color="red", linestyle="--", label="No Error")
plt.legend()
plt.show()

In [ ]:
#Biggest error list
errors_df = pd.DataFrame({
    "Actual": y_test,
    "Predicted": voting_pred,
    "Residual": residuals
})

largest_errors = errors_df.sort_values(by="Residual", key=abs, ascending=False).head(10)
print("Top 10 Instances with Largest Errors:")
print(largest_errors)

## 5.3 Stacking

### Train & Tune

In [ ]:
estimators = [
    ("gbr", gbr_model_ensembles),
    ("rfr", rfr_model_ensembles),
    ('hgb', hgb_model_ensembles), 
    ('ada', ada_model_ensembles) 
    
]

stacking_model = StackingRegressor(
    estimators = estimators, 
    final_estimator = xgb_model_ensembles
)

stacking_model.fit(X_train_selected, y_train)

In [ ]:
stacking_score = cross_val_score(stacking_model, X_train_selected, y_train, scoring=make_scorer(r2_score), cv=5)
print("Cross-validated R²:", stacking_score.mean())

### Test & Evaluate

In [ ]:
stacking_pred = stacking_model.predict(X_test_selected)

In [ ]:
score = stacking_model.score(X_test_selected, y_test)
mse = mean_squared_error(y_test, stacking_pred)
mae = mean_absolute_error(y_test, stacking_pred)
r2 = r2_score(y_test, stacking_pred)
rmse = np.sqrt(mse)

print("Model Evaluation Metrics:")
print(f"Score: {score:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

In [ ]:
#Actual vs predicted
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=stacking_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="Perfect Fit")
plt.title("Actual vs Predicted Values")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.legend()
plt.show()

In [ ]:
#Residual histogram
residuals = y_test - stacking_pred

plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, bins=30, color="blue")
plt.title("Residual Distribution")
plt.xlabel("Residuals (Actual - Predicted)")
plt.ylabel("Frequency")
plt.axvline(0, color="red", linestyle="--", label="No Error")
plt.legend()
plt.show()

In [ ]:
#Biggest error list
errors_df = pd.DataFrame({
    "Actual": y_test,
    "Predicted": stacking_pred,
    "Residual": residuals
})

largest_errors = errors_df.sort_values(by="Residual", key=abs, ascending=False).head(10)
print("Top 10 Instances with Largest Errors:")
print(largest_errors)

# 6. Model Analysis

## 6.1 Clean Model

In [ ]:
# Decide what model to train? 
xgb_model_trial = XGBRegressor(colsample_bytree= 0.7, learning_rate= 0.05, max_depth= 5, n_estimators= 500, subsample= 1.0)

xgb_model_trial.fit(X_train_selected,y_train)

## 6.2 Permutation Importance

In [ ]:
pi = permutation_importance(
    xgb_model_trial,            
    X_test_selected,          
    y_test,          
    n_repeats=30,
    scoring='r2'
)

In [ ]:
# get feature names

In [ ]:
pi_df = pd.DataFrame({
    'feature': selected_features,
    'pi_mean': pi.importances_mean,
    'pi_std': pi.importances_std
}).sort_values(by='pi_mean', ascending=True)

In [ ]:
pi_df

In [ ]:
plt.figure(figsize=(10, 6))
plt.errorbar(
    pi_df['pi_mean'],
    pi_df['feature'],
    xerr=pi_df['pi_std'],
    fmt='o', color='black', ecolor='lightgray', elinewidth=3, capsize=0
)
plt.xlabel("Permutation Importance Mean (Drop in R²)")
plt.ylabel("Feature")
plt.title("Permutation Importance (RandomForestRegressor)")
plt.tight_layout()
plt.show()

## 6.3 SHAP

In [ ]:
def shap_func(model, X_train, X_test, model_name):
    
    #if model uses ttr
    if hasattr(model, 'regressor_'):
        base_model = model.regressor_
    else:
        base_model = model
        
    explainer = shap.Explainer(base_model, X_train)
    shap_values = explainer(X_test, check_additivity=False)
    
    print(f"\nSHAP Summary for {model_name}")
    shap.summary_plot(shap_values, features=X_test, plot_type="bar", show=False)
    plt.title(f"SHAP Feature Importance ({model_name})")
    plt.tight_layout()
    plt.show()
    
    shap.plots.beeswarm(shap_values)
    
    row_idx = 0  # or any row index, to explain single prediction
    shap.plots.waterfall(shap_values[row_idx])
    
    shap_df = pd.DataFrame(shap_values.values, columns=selected_features)

    mean_abs_shap = shap_df.abs().mean().sort_values(ascending=False)
    print(f"\nTop SHAP Features for {model_name}:\n", mean_abs_shap.head(10))

    return shap_values, mean_abs_shap

In [ ]:
shap_rfr, shap_rfr_importance = shap_func(rfr_model_ensembles, X_train_selected, X_test_selected, model_name=" RFR ")

In [ ]:
shap_gbr, shap_gbr_importance = shap_func(gbr_model_ensembles, X_train_selected, X_test_selected, model_name=" GBR ")

In [ ]:
shap_hgb, shap_hgb_importance = shap_func(hgb_model_ensembles, X_train_selected, X_test_selected, model_name=" HGB ")

In [ ]:
shap_xgb, shap_xgb_importance = shap_func(xgb_model_ensembles, X_train_selected, X_test_selected, model_name=" XGBoost ")

In [ ]:
# shap_ada, shap_ada_importance = shap_func(ada_model_ensembles, X_train_selected, X_test_selected, model_name=" ADA ")

## 6.4 PDP

In [ ]:
X_train_selected_df = pd.DataFrame(X_train_selected, columns=selected_features)
X_test_selected_df = pd.DataFrame(X_test_selected, columns=selected_features)

In [ ]:
#can change to whatever is the best model
top_features = shap_xgb_importance.head(8).index.tolist()
print(top_features)

In [ ]:
def plot_pdp(model, X, features, model_name="Model"):
    print(f"Plotting PDPs for {model_name}...")
    fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)
    
    PartialDependenceDisplay.from_estimator(
        model,
        X,
        features=features,
        kind="both",                   
        subsample=100,
        grid_resolution=30,
        n_cols=2,
        random_state=0,
        ax=ax,
        pd_line_kw={"color": "red"}   
    )
    
    plt.suptitle(f"PDPs for {model_name}", y=1.02)
    plt.show()

In [ ]:
rfr_model_pdp = RandomForestRegressor(bootstrap = True, 
                                            max_depth = 15, 
                                            max_features = 'sqrt', 
                                            min_samples_leaf = 4, 
                                            min_samples_split = 15, 
                                            n_estimators = 1600 )

rfr_model_pdp.fit(X_train_selected,y_train)

In [ ]:
selected_features

In [ ]:
plot_pdp(rfr_model_pdp, X_test_selected_df, selected_features, model_name=" RFR ")

In [ ]:
gbr_model_pdp = GradientBoostingRegressor(subsample = 1.0,
                                                n_estimators = 400,
                                                min_samples_split = 5,
                                                min_samples_leaf = 2, 
                                                max_features = 'sqrt',
                                                max_depth = 10, 
                                                learning_rate = 0.01)
gbr_model_pdp.fit(X_train_selected,y_train)

In [ ]:
plot_pdp(gbr_model_pdp, X_test_selected_df, selected_features, model_name=" GBR ")

In [ ]:
hgb_model_pdp = HistGradientBoostingRegressor(learning_rate = 0.1, 
                                                    max_depth = 30, 
                                                    max_iter = 600, 
                                                    max_leaf_nodes = 50, 
                                                    min_samples_leaf = 20 )

hgb_model_pdp.fit(X_train_selected,y_train)

In [ ]:
plot_pdp(hgb_model_pdp, X_test_selected_df, selected_features, model_name=" HGB ")

In [ ]:
xgb_model_pdp = XGBRegressor(verbosity=0, 
                                   colsample_bytree = 0.9, 
                                   learning_rate = 0.05, 
                                   max_depth = 7, 
                                   n_estimators = 300, 
                                   subsample = 1.0)

xgb_model_pdp.fit(X_train_selected,y_train)

In [ ]:
plot_pdp(xgb_model_pdp, X_test_selected_df, selected_features, model_name=" XGB ")

In [ ]:
ada_model_pdp = AdaBoostRegressor(learning_rate = 0.005, 
                                        loss = 'linear', 
                                        n_estimators = 100)

ada_model_pdp.fit(X_train_selected,y_train)

In [ ]:
plot_pdp(ada_model_pdp, X_test_selected_df, selected_features, model_name=" ADA ")